In [1]:
from datetime import date, datetime, timedelta

import pandas as pd
import numpy as np

import random
import copy
import math

# Read dataset place

In [2]:
df_places = pd.read_csv('../Dataset/places.csv')
df_places.head()

,id,name,type,latitude,longitude,tarif,durasi,rating
0,1,MALIOBORO JOGJAKARTA,location,-7.792576,110.365843,0,5400,4.8
1,2,Tourism Zone Malioboro,location,-7.798249,110.365279,0,3600,4.7
2,3,Plengkung Gading,location,-7.813728,110.362907,0,1800,4.6
3,4,Bringin Kembar Yogyakarta,location,-7.811936,110.363194,0,5400,4.6
4,5,Rumah Hantu Malioboro,location,-7.795638,110.365354,35000,2700,4.5


In [3]:
df_places.type.unique()

array(['location', 'hotel'], dtype=object)

In [4]:
df_places.dtypes

id             int64
name          object
type          object
latitude     float64
longitude    float64
tarif          int64
durasi         int64
rating       float64
dtype: object

# Read dataset timematrix

In [5]:
df_time_matrix = pd.read_csv('../Dataset/place_timematrix.csv')
df_time_matrix.head()

,id,id_a,id_b,durasi
0,1,1,1,0
1,2,1,2,134
2,3,1,3,605
3,4,1,4,595
4,5,1,5,79


In [6]:
df_time_matrix.dtypes

id        int64
id_a      int64
id_b      int64
durasi    int64
dtype: object

# Read dataset schedule

In [7]:
df_schedule = pd.read_csv('../Dataset/place_jadwal.csv')
df_schedule.head()

,id,id_tempat,jam_buka,jam_tutup,hari
0,1,1,00:00,23:59,senin
1,2,2,00:00,23:59,senin
2,3,3,00:00,23:59,senin
3,4,4,00:00,23:59,senin
4,5,5,16:00,18:00,senin


In [8]:
df_schedule.dtypes

id            int64
id_tempat     int64
jam_buka     object
jam_tutup    object
hari         object
dtype: object

In [9]:
df_schedule['jam_buka'] = df_schedule['jam_buka'].apply(lambda x: int(x[:2]) * 3600 + int(x[3:]) * 60)
df_schedule['jam_tutup'] = df_schedule['jam_tutup'].apply(lambda x: int(x[:2]) * 3600 + int(x[3:]) * 60)
df_schedule.head()

,id,id_tempat,jam_buka,jam_tutup,hari
0,1,1,0,86340,senin
1,2,2,0,86340,senin
2,3,3,0,86340,senin
3,4,4,0,86340,senin
4,5,5,57600,64800,senin


In [10]:
df_schedule.dtypes

id            int64
id_tempat     int64
jam_buka      int64
jam_tutup     int64
hari         object
dtype: object

# Parameters

In [11]:
AGENT_COUNT = 10
MAX_ITERATIONS = 10
HOTEL_ID = 111
DEPART_TIME = 8 * 3600
ARRIVAL_TIME = 21 * 3600
START_DATE = date(year=2023, month=10, day=1)
SELECTED_POI = [1, 6, 7, 8, 12, 37, 39, 45, 57, 61, 65, 72, 77, 94, 95]
AGENT_LENGTH = len(SELECTED_POI)
DAYS_COUNT = 3

DOI_DURATION = 0.3
DOI_COST = 0.2
DOI_RATING = 0.5
DOI_POI_INCLUDED = 1
TOTAL_DOI = DOI_DURATION + DOI_COST + DOI_RATING + DOI_POI_INCLUDED
DOI_DURATION = DOI_DURATION / TOTAL_DOI
DOI_COST = DOI_COST / TOTAL_DOI
DOI_RATING = DOI_RATING / TOTAL_DOI
DOI_POI_INCLUDED = DOI_POI_INCLUDED / TOTAL_DOI

# Functions

Function to get pois cost

In [82]:
def get_pois_cost(_id):
    return df_places[df_places['id'].isin(_id)]['tarif'].tolist()
get_pois_cost([4, 5, 9])

[0, 35000, 0]

Function to get pois rating

In [77]:
def get_pois_rating(_ids):
    return df_places[df_places['id'].isin(_ids)]['rating'].tolist()
get_pois_rating([4, 5, 9])

[4.6, 4.5, 4.7]

Function to get min rating

In [14]:
def get_min_rating():
    min_series = df_places.min(numeric_only=True)
    return min_series['rating']

min_rating = get_min_rating()
min_rating

3.7

Function to get max rating

In [15]:
def get_max_rating():
    max_series = df_places.max(numeric_only=True)
    return max_series['rating']

max_rating = get_max_rating()
max_rating

5.0

Function to get average rating

In [16]:
def get_average_rating(_ids):
    return df_places[df_places['id'].isin(_ids)].mean(numeric_only=True)['rating']

Function to get min cost

In [17]:
def get_min_cost():
    return 0

Function to get max cost

In [18]:
def get_cost(selected_agents):
    return df_places[df_places['id'].isin(selected_agents)].sum()['tarif']

max_cost = get_cost([1, 4, 6, 45])
max_cost

11000

Function to get poi duration

In [19]:
def get_poi_duration(_id):
    return df_places[df_places['id'] == _id].iloc[0]['durasi']

duration = get_poi_duration(2)
duration

3600

Function to get poi opening hour

In [20]:
def get_poi_opening_hour(_id):
    return df_schedule[df_schedule['id_tempat'] == _id].iloc[0]['jam_buka']

Function to get poi closing hour

In [21]:
def get_poi_closing_hour(_id):
    return df_schedule[df_schedule['id_tempat'] == _id].iloc[0]['jam_tutup']

Function to get travel time between 2 places

In [22]:
def get_travel_time(_id1, _id2):
    df_filter = df_time_matrix[(df_time_matrix['id_a'] == _id1) & (df_time_matrix['id_b'] == _id2)]
    return df_filter.iloc[0]['durasi']

get_travel_time(5, 44)

1109

Function to generate initial population

In [23]:
def generate_initial_population(agent_count, agent_length):
    return [[random.random() * 10 for i in range(agent_length)] for j in range(agent_count)]

agents = generate_initial_population(AGENT_COUNT, AGENT_LENGTH)
len(agents[0])

15

Function to get day with fewest poi assigned to it 

In [24]:
def get_day_with_fewest_poi(routes, index_ignored):
    selected_index = 0
    min_poi_assigned = 200
    for i in range(len(routes)):
        if i not in index_ignored:
            if len(routes[i]) < min_poi_assigned:
                selected_index = i
                min_poi_assigned = len(routes[i])
    return selected_index

Function to check total duration in single day

In [25]:
def get_single_day_duration(single_day_route):
    if len(single_day_route) == 0:
        return 0

    if len(single_day_route) == 1:
        duration = get_travel_time(HOTEL_ID, single_day_route[0])
        duration += get_poi_duration(single_day_route[0])
        duration += get_travel_time(single_day_route[0], HOTEL_ID)
        return duration
    
    duration = 0
    for i in range(len(single_day_route)):
        if i == 0: # first poi
            duration += get_travel_time(HOTEL_ID, single_day_route[i])
            duration += get_poi_duration(single_day_route[i])
        elif i != len(single_day_route)-1:
            duration += get_travel_time(single_day_route[i], single_day_route[i+1])
            duration += get_poi_duration(single_day_route[i+1])
        else: # last poi
            duration += get_travel_time(single_day_route[i], HOTEL_ID)
    
    return duration

Function to check total multi day itinerary duration

In [26]:
def get_multi_day_duration(routes):
    durations = 0
    for route in routes:
        durations += get_single_day_duration(route)
    return durations

Function to check if poi is able to be assigned to a single day

In [27]:
def check_poi_able_to_be_assigned(single_day_route, poi_id):
    sdr = copy.deepcopy(single_day_route)
    sdr.append(poi_id)
    
    last_poi_id = sdr[len(sdr)-1]
    single_day_duration = get_single_day_duration(sdr)
    single_day_duration -= get_travel_time(last_poi_id, HOTEL_ID)
    single_day_duration += DEPART_TIME
    
    arrival_time_to_poi = single_day_duration + get_travel_time(last_poi_id, poi_id)
    departure_time_from_poi = arrival_time_to_poi + get_poi_duration(poi_id)
    arrival_time_to_hotel = departure_time_from_poi + get_travel_time(poi_id, HOTEL_ID)

    if arrival_time_to_hotel > ARRIVAL_TIME:
        return False

    if departure_time_from_poi > get_poi_closing_hour(poi_id):
        return False

    if arrival_time_to_poi < get_poi_opening_hour(poi_id):
        return False
    
    return True

check_poi_able_to_be_assigned([1], 4)

True

Function to check if any poi able to assign

In [28]:
def check_any_poi_able_to_be_assigned(routes, R, R_assigned):
    for _id in R:
        if _id not in R_assigned:
            for route in routes:
                if check_poi_able_to_be_assigned(route, _id):
                    return True
    return False

Function to split route with greedy algorithm

In [29]:
def greedy_separate_route(agent, selected_poi, days_count):
    A = np.argsort(agent)
    R = [selected_poi[A[i]] for i in range(len(selected_poi))]
    R_assigned = []
    
    routes = [[] for _ in range(days_count)] # for storing route for each day
    is_any_poi_able_to_assign = check_any_poi_able_to_be_assigned(routes, R, R_assigned)
    while is_any_poi_able_to_assign: # if any POI able to be assigned
        days_included = []
        selected_day = get_day_with_fewest_poi(routes, days_included)
        poi_has_assigned = False
        while not poi_has_assigned:
            i = 0
            while i < len(R) and not poi_has_assigned:
                if check_poi_able_to_be_assigned(routes[selected_day], R[i]) and R[i] not in R_assigned:
                    routes[selected_day].append(R[i])
                    R_assigned.append(R[i])
                    poi_has_assigned = True
                i += 1
            if not poi_has_assigned:
                days_included.append(selected_day)
                selected_day = get_day_with_fewest_poi(routes, days_included)
        is_any_poi_able_to_assign = check_any_poi_able_to_be_assigned(routes, R, R_assigned)
    
    return routes

agents = generate_initial_population(AGENT_COUNT, AGENT_LENGTH)
route_days = greedy_separate_route(agents[0], SELECTED_POI, DAYS_COUNT)
route_days

[[1, 95, 61, 7], [57, 45, 77, 65], [94, 72, 37, 39]]

Function to get v value

In [30]:
def get_v(value, min, max, is_greater_better):
    if value < min:
        if is_greater_better:
            return 0
        return 1
    if value > max:
        if is_greater_better:
            return 1
        return 0
    return (abs(value - min) / (max - min)) if is_greater_better else (abs(max - value) / (max - min))

Set parameter for fitness function

In [31]:
duration_range = [0, 100000]
rating_range = [get_min_rating(), get_max_rating()]
cost_range = [get_min_cost(), get_cost(SELECTED_POI)]
poi_included_range = [1, len(SELECTED_POI)]

Fitness function

In [32]:
def fitness_function(agent):
    routes = greedy_separate_route(agent, SELECTED_POI, DAYS_COUNT)
    
    duration = get_multi_day_duration(routes)

    assigned_ids = []
    for day_route in routes:
        assigned_ids.extend(day_route)
    # df_temp = df_places[df_places['id'].isin(assigned_ids)]
    
    rating = get_average_rating(assigned_ids)
    costs = get_cost(assigned_ids)
    
    MAUT = 0
    MAUT += get_v(duration, duration_range[0], duration_range[1], False)
    MAUT += get_v(rating, rating_range[0], rating_range[1], True)
    MAUT += get_v(costs, cost_range[0], cost_range[1], False)
    MAUT += get_v(len(assigned_ids), poi_included_range[0], poi_included_range[1], True)
    return MAUT

agents = generate_initial_population(AGENT_COUNT, AGENT_LENGTH)
fitness_value = fitness_function(agents[8])
fitness_value

1.8322062008492974

Function to get best agent

In [33]:
def get_best_agent(agents):
    index = -1
    max = 0
    i = 0
    for agent in agents:
        fitness_value = fitness_function(agent)
        if fitness_value > max:
            max = fitness_value
            index = i
        i += 1
    return (max, agents[index])

agents = generate_initial_population(AGENT_COUNT, AGENT_LENGTH)
fitness_value, best_agent = get_best_agent(agents)
fitness_value

2.4700038208693953

WOA function

In [34]:
def WOA(min_x, max_x, agents):
    t = 0

    # Fbest : nilai fitness terbaik
    # Xbest : agen terbaik
    Fbest, Xbest = get_best_agent(agents)
    agent_dimension = int(len(Xbest) / DAYS_COUNT) # banyak tempat wisata sesuai preferensi user
    fitness_values = []

    # Menyimpan nilai fitness untuk setiap agen
    for i in range(len(agents)):
        fitness_values.append(fitness_function(agents[i]))

    # print("Initial best fitness = %.5f" % Fbest)
    while t < MAX_ITERATIONS:

        a = 2 * (1 - t / MAX_ITERATIONS)
        a2 = -1 + t * ((-1)/MAX_ITERATIONS)

        i = 0
        for agent in agents:
            A = 2 * a * random.random() - a
            C = 2 * random.random()
            b = 1
            l = (a2-1)*random.random()+1

            D = [0.0 for k in range(agent_dimension)]
            D1 = [0.0 for k in range(agent_dimension)]
            Xnew = [0.0 for k in range(agent_dimension)]
            Xrand = [0.0 for k in range(agent_dimension)]
            p = random.random()
            if p < 0.5:
                if abs(a) >= 1: # search for prey
                    p = random.randint(0, AGENT_COUNT-1)
                    while (p==i):
                        p = random.randint(0, AGENT_COUNT-1)

                    Xrand = agents[p]

                    for j in range(agent_dimension):
                        D[j] = abs(C * Xrand[j] - agent[j])
                        Xnew[j] = Xrand[j] - A * D[j]
                else: # encircling prey
                    for j in range(agent_dimension):
                        D[j] = abs(C * Xrand[j] - agent[j])
                        Xnew[j] = Xrand[j] - A * D[j]
            else: # bubble net attacking
                for j in range(agent_dimension):
                    D1[j] = abs(Xbest[j] - agent[j])
                    Xnew[j] = D1[j] * math.exp(b * l) * math.cos(2 * math.pi * l) + Xbest[j]

            for j in range(agent_dimension):
                agent[j] = Xnew[j]
            i += 1

        for i in range(len(agents)):
            # jika Xnew < minx atau Xnew > maxx
            for j in range(agent_dimension):
                agents[i][j] = max(agents[i][j], min_x)
                agents[i][j] = min(agents[i][j], max_x)

            fitness_values[i] = fitness_function(agents[i])

            if (fitness_values[i] > Fbest):
                Xbest = copy.copy(agents[i])
                Fbest = fitness_values[i]

        # print("Iteration = " + str(t) + " | best fitness = %.5f" % Fbest)
        t += 1
    return Xbest, Fbest

agents = generate_initial_population(AGENT_COUNT, AGENT_LENGTH)
agent, fit_value = WOA(0, 10, agents)
print('agent')
print(agent)
print('Fitness value : ', fit_value)

agent
[0, 0, 0, 0, 0, 5.373518046554518, 1.3245565102207546, 2.145205159702279, 3.1849282215822496, 6.274371079422022, 6.999367497054142, 9.300155223169046, 2.6975734877643323, 3.410658388783413, 3.24599910154191]
Fitness value :  2.5175081964937713


Function to get time line

In [35]:
def get_time_line(l):
    if len(l) == 0:
        return []
    current_time = datetime(2023, 11, 19, 8, 0, 0)
    time_line = [current_time.strftime('%H:%M:%S')]

    i = 0
    while i < len(l):
        if i == 0:
            travel_time = get_travel_time(HOTEL_ID, l[i])
            time_delta = timedelta(seconds=np.int16(travel_time).item())
            current_time += time_delta
            time_line.append(current_time.strftime('%H:%M:%S'))
        elif i != len(l)-1:
            time_spent = get_poi_duration(l[i])
            travel_time = get_travel_time(l[i], l[i+1])
            time_delta = timedelta(seconds=np.int16(travel_time).item() + np.int16(time_spent).item())
            current_time += time_delta
            time_line.append(current_time.strftime('%H:%M:%S'))
        else:
            time_spent = get_poi_duration(l[i])
            time_delta = timedelta(seconds=np.int16(time_spent).item())
            current_time += time_delta
            time_line.append(current_time.strftime('%H:%M:%S'))

            travel_time = get_travel_time(l[i], HOTEL_ID)
            time_delta = timedelta(seconds=np.int16(travel_time).item())
            current_time += time_delta
            time_line.append(current_time.strftime('%H:%M:%S'))
        i += 1

    return time_line

Function to construct solution, output best agent

In [83]:
def construct_solution():
    #generate initial population
    agents = generate_initial_population(AGENT_COUNT, AGENT_LENGTH)

    agents_ = copy.deepcopy(agents)
    Xbest, Fbest = WOA(0.0, 10.0, agents_)
    route = greedy_separate_route(Xbest, SELECTED_POI, DAYS_COUNT)
    fitness_value = fitness_function(Xbest)
    # print(f'Fitness value : ${fitness_value}')
    duration = get_multi_day_duration(route)
    # print('Route : ', route)

    output = {'results': []}
    for day_route in route:
        output['results'].append({
            'index': day_route,
            'waktu': get_time_line(day_route),
            'rating': get_pois_rating(day_route),
            'tarif': get_pois_cost(day_route),
        })

    return output , Fbest

In [84]:
output, Fbest = construct_solution()

In [85]:
output

{'results': [{'index': [1, 77, 37, 61],
   'waktu': ['08:00:00',
    '08:04:00',
    '10:07:17',
    '11:35:24',
    '12:35:24',
    '12:44:06'],
   'rating': [4.8, 4.5, 4.3, 4.6],
   'tarif': [0, 5000, 0, 0]},
  {'index': [57, 65, 94, 95],
   'waktu': ['08:00:00',
    '08:11:28',
    '11:17:18',
    '13:01:59',
    '13:46:59',
    '14:07:44'],
   'rating': [4.5, 4.6, 4.3, 4.3],
   'tarif': [0, 10000, 0, 0]},
  {'index': [7, 45, 39],
   'waktu': ['08:00:00', '08:07:38', '10:27:49', '11:27:49', '12:28:44'],
   'rating': [4.8, 4.6, 4.7],
   'tarif': [0, 6000, 8000]}]}

In [86]:
Fbest

2.5376581964937714